In [2]:
from py_ecc.bn128 import G1, multiply, add, FQ, eq, Z1
from py_ecc.bn128 import curve_order
import numpy as np
from functools import reduce
import random
import galois

In [3]:
import sys
sys.path.append('../hw_1')
from pedersen_commitment import generate_n_random_points, pedersen_commitment

In [4]:
GF = galois.GF(curve_order)

In [5]:
def gf_inverse(a):
    return GF(1) / a

In [6]:
def random_element():
    return random.randint(2, curve_order)

def add_points(*points):
    return reduce(add, points, Z1)

In [7]:
# if points = G1, G2, G3, G4 and scalars = a,b,c,d vector_commit returns
# aG1 + bG2 + cG3 + dG4
def vector_commit(points, scalars):
    # return reduce(add, [multiply(P, i) for P, i in zip(points, scalars)], Z1)
    result = Z1
    for P, i in zip(points, scalars):
        result = add(result, multiply(P, int(i)))
    return result

In [8]:
# return a folded vector of length n/2 for scalars
def fold(scalar_vec, u):
    # if scalar_vec is odd add 0 to the end
    if len(scalar_vec) % 2 == 1:
        scalar_vec.append(0)
    # fold the vector
    return [scalar_vec[i] * u + gf_inverse(u) * scalar_vec[i + 1] for i in range(0, len(scalar_vec), 2)]


In [9]:
# return a folded vector of length n/2 for points
def fold_points(point_vec, u):
    if len(point_vec) % 2 == 1:
        point_vec.append(Z1)
    # fold the vector
    return [add(multiply(point_vec[i], int(u)), multiply(point_vec[i + 1], int(gf_inverse(u)))) for i in range(0, len(point_vec), 2)]


In [10]:
def compute_secondary_diagonal(G_vec, a):
    if len(a) % 2 == 1:
        a.append(0)
        G_vec.append(Z1)
    
    L = vector_commit([G_vec[i+1] for i in range(0, len(G_vec)-1, 2)], [a[i] for i in range(0, len(a)-1, 2)])
    R = vector_commit([G_vec[i] for i in range(0, len(G_vec)-1, 2)], [a[i+1] for i in range(0, len(a)-1, 2)])
    
    return L, R

In [11]:
def verify(A, L, R, a_prime, u, G_vec):
    # Calculate right hand side: L·u² + A + R·u⁻²
    u_squared = int((u * u))
    u_inv_squared = int((gf_inverse(u) * gf_inverse(u)))
    
    rhs = multiply(L, u_squared)  # Start with L·u²
    rhs = add(rhs, A)            # Add A
    rhs = add(rhs, multiply(R, u_inv_squared))  # Add R·u⁻²

    # Calculate left hand side
    # <folded_g, a_prime>
    folded_G = fold_points(G_vec, gf_inverse(u))
    lhs = vector_commit(folded_G, a_prime)

    return eq(lhs, rhs)

In [12]:
a = [9, 10, 11, 12]
G_vec = generate_n_random_points('hello', 2**4)

# pad a with 0s to make the length powers of 2
while len(a) < 2**4:
    a.append(0)

while len(a) > 1:
  A = vector_commit(G_vec, a)
  L, R = compute_secondary_diagonal(G_vec, a)

  u = GF(random_element())
  a = fold(a, u)
  
  if not verify(A, L, R, a, u, G_vec):
    print("Verification failed")
    break
  G_vec = fold_points(G_vec, gf_inverse(u))

A = vector_commit(G_vec, a)


False
